In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point
import geopandas as gpd
import contextily as ctx
from config import create_connection_obj

pd.set_option('display.max_columns', None)
sns.set_theme(style="whitegrid")

In [2]:
conn = create_connection_obj()

rawdata = pd.read_sql(sql="SELECT * FROM raw_redfin_data;", con=conn)
recent_sales = pd.read_sql(sql="SELECT * FROM last_weeks_sales;", con=conn)

In [3]:
def process_raw_data(df):
    """Filter on relevant observations, convert to GeoDataFrame, and drop uncessary columns

    Args:
        df (pd.DataFrame): Raw Redfin data queries from db
    Output:
        df (gpd.GeoDataFrame)
    """
    to_drop = [
        'SALE_TYPE',
        'PROPERTY_TYPE',
        'STATE',
        'LOT_SIZE',
        'LOCATION',
        'DAYS_ON_MARKET',
        'STATUS',
        'NEXT_OPEN_HOUSE_START_TIME',
        'NEXT_OPEN_HOUSE_END_TIME',
        'URL',
        'SOURCE',
        'MLS',
        'FAVORITE',
        'INTERESTED',
        'CITY',
        'SOLD_DATE'
        ]

    def convert_to_geodataframe(df_):
        coords = list(zip(df_['LONGITUDE'], df_['LATITUDE']))
        geos = [Point(xy) for xy in coords]
        return gpd.GeoDataFrame(df_, geometry=geos, crs="EPSG:4269")

    return (
        df
        .query("PROPERTY_TYPE == 'Condo/Co-op' & CITY == 'Portland'")
        .pipe(convert_to_geodataframe)
        .astype({'SOLD_DATE':'datetime64'})
        .assign(
            SOLD_YEAR=lambda df_: df_['SOLD_DATE'].dt.year,
            SOLD_MONTH=lambda df_: df_['SOLD_DATE'].dt.month
            )
        .drop(to_drop, axis=1)
        )
    
    
def breakout_address_features(df):
    '''
    TBD
    '''
    df = pd.concat(
        [
            df, 
            df
            ['ADDRESS']
            .str.extract('(^\d{,5} )(NE|NW|SE|SW|N|E|S|W)(.+)', expand=True)
            .rename(columns={0:'address_number', 1:'address_direction', 2:'address_street'})
        ], axis=1
    )
    
    return df


def estimate_unit_floor(df):
    if df['unit_len'] <= 2:
        return 1
    elif df['unit_len'] == 3:
        return int(df['unit_number'][0])
    elif df['unit_len'] == 4:
        return int(df['unit_number'][:1])


def assign_unit_floor(df):
    '''
    TBD
    '''
    df = pd.concat([
        df, (
        df
        ['address_street']
        .str.lstrip()
        .str.split(' ', expand=True)
        .rename(columns={0:'tmp1', 1:'tmp2', 2:'unit_number'})
        .assign(street_name=lambda df_:df_['tmp1'] + ' ' + df_['tmp2'])
        .assign(
            unit_number=lambda df_: np.where(df_['unit_number'].str.replace('#', '').str.contains('\d'), df_['unit_number'].str.replace('#', ''), '0')
        )
        .assign(
            unit_number=lambda df_: df_['unit_number'].fillna('0'),
            unit_len=lambda df_: df_['unit_number'].str.len()
            )
        .assign(unit_floor=lambda df_: df_.apply(estimate_unit_floor, axis=1))
        [['street_name', 'unit_floor']]
        )
    ], axis=1)
    
    return df


def extract_address_building(df):
    return (
        df
        .assign(
            address_building=df['address_number'] + df['address_direction'] + ' ' + df['street_name']
            )
        .drop(['address_number', 'address_street', 'street_name'], axis=1)
    )


def extract_features(df):
    '''
    TBD
    '''
    return (
        df
        .pipe(breakout_address_features)
        .pipe(assign_unit_floor)
        .pipe(extract_address_building)
    )


def prep_data_for_modeling(df):
    '''
    TBD
    '''
    return (
        df
        .pipe(process_raw_data)
        .pipe(extract_features)
    )


In [4]:
(
    pd.concat([
        prep_data_for_modeling(rawdata), 
        prep_data_for_modeling(recent_sales
                              )], axis=0)
    .drop(['geometry'], axis=1)
    .to_sql(con=conn, name='tidyredfin_data', if_exists='replace')
)